# Data Preprocessing Version 3

Data Preprocessing:
1. Segment Selection and Matching:
    - select target segments for input and output
    - match upstream and downstream segments 
2. Input and Output:
    - filter by segments and dates
    - speed processing
    - incident processing
        - mark as incident based on (1) Waze & RCRS report and (2) slow down speed
        - mark as incident based on abnormal speed
        - for output, also mark as incident based on upstream abnormal speed
    - concat input features together and normalize

Key Differences from Data Preprocessing Version 1 & 2:
- Input:
    - Based on the additional information about the mapping between TMC & XD, the target segements selected here are different.
- Output Ground Truth
    - new_Y here includes speed ground truth of both TMC & XD, as well as incident status data



In [1]:
import csv
import pandas as pd
import numpy as np
# import geopandas as gpd
import networkx as nx
import pickle

from datetime import datetime as dt
from collections import Counter
from scipy.spatial.distance import cdist
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm


In [2]:
pwd

'/Users/haowu/Desktop/Transportation Research/Traffic-Prediction/pipeline_v2/data_preprocessing'

In [3]:
cd ..

/Users/haowu/Desktop/Transportation Research/Traffic-Prediction/pipeline_v2


## Part 1. Segment Selection for New Input & Output

### 1.0 Read Files of Segment IDS (TMC & XD)

In [4]:
'''
Columns:
    'tmc', 'road', 'direction', 'intersection', 'state', 'county', 'zip',
    'start_latitude', 'start_longitude', 'end_latitude', 'end_longitude',
    'miles', 'road_order', 'timezone_name', 'type', 'country', 'tmclinear',
    'frc', 'border_set', 'f_system', 'urban_code', 'faciltype', 'structype',
    'thrulanes', 'route_numb', 'route_sign', 'route_qual', 'altrtename',
    'aadt', 'aadt_singl', 'aadt_combi', 'nhs', 'nhs_pct', 'strhnt_typ',
    'strhnt_pct', 'truck', 'isprimary', 'active_start_date',
    'active_end_date'
'''
tmc = pd.read_csv("../data/Carnberry_NPMRDS_5min/manually_select_cranberry_2019_dont_average_2/TMC_Identification.csv")  # (1248, 39), this is the comprehensive list, TMC speed csv data (All/Truck/PV) may not include all TMC in this list
'''
Columns:
    'xd', 'road-name', 'road-num', 'bearing', 'miles', 'frc', 'county',
    'state', 'zip', 'timezone_name', 'start_latitude', 'start_longitude',
    'end_latitude', 'end_longitude'
'''
xd = pd.read_csv("../data/Cranberry_ritis_1min_class123/manually_select_cranberry_class123_20181101_20190727_dont_average/XD_Identification.csv")  # (1628, 14)
xd["id_xd_str"] = xd["xd"].apply(str)
xd["id_xd_int"] = xd["xd"].apply(int)

'''
162 targeted segments from old output
    TMC: 129 (84 found in tmc)
    XD: 33 (21 found in xd)
'''
old_out = list(np.load("../data/cran_tmc.npy", allow_pickle=True))
old_out_tmc = old_out[:129]
old_out_xd = old_out[130:]


col_names = list(np.load("../data/col_names.npy", allow_pickle=True))
'''
369 segments from old input
    Speed (tti): (369 segments)
        TMC: 331
        XD: 38
    Incident: (315 segments)
        TMC: 275
        XD: 40
    SD: (303 segments)
        TMC: 266
        XD: 37
'''
old_in_seg = {}
old_in_seg["tti"] = {} # 369 (331 tmc + 38 xd), including all 162 segments in old_target
old_in_seg["inc"] = {} # 369 (331 tmc + 38 xd), including all 162 segments in old_target
old_in_seg["tti"]["tmc"] = [i for i in col_names[:-21] if "sd" not in i and "inc" not in i and i.startswith("104")]
old_in_seg["tti"]["xd"] = [i for i in col_names[:-21] if "sd" not in i and "inc" not in i and not i.startswith("104")]
old_in_seg["inc"]["tmc"] = [i[4:] for i in col_names[:-21] if "inc" in i and i[4:].startswith("104")]  # 275 tmc 
old_in_seg["inc"]["xd"] = [i[4:] for i in col_names[:-21] if "inc" in i and not i[4:].startswith("104")] # 40 xd
old_in_seg["sd"] = [i[3:] for i in col_names[:-21] if "sd" in i] # 303 (266 tmc + 37 xd), subset of old_in_seg["inc"]

### 1.1 Match TMC and XD Segments

In [9]:
tmc_xd_join = pd.read_csv("../data/tmc_xd_join.csv") # 146 tmc segments to 260 xd segments; contains all new_out_tmc and new_in_tmc
tmc_xd_join = tmc_xd_join[(tmc_xd_join.id_xd_int.isin(xd.id_xd_int)) & (tmc_xd_join.id_tmc.isin(tmc.tmc))]

# # construct and save dictionary to map tmc segments to/from xd segments
# tmc_to_xd = tmc_xd_join.groupby("id_tmc")["id_xd_int"].apply(list).reset_index()
# tmc_to_xd = (tmc_to_xd.set_index("id_tmc")).to_dict(orient="index") # 146 tmc segments, key: <id_tmc>; value: dict() with key "id_xd_int"

# xd_to_tmc = tmc_xd_join.groupby("id_xd_int")["id_tmc"].apply(list).reset_index()
# xd_to_tmc = (xd_to_tmc.set_index("id_xd_int")).to_dict(orient="index") # 260 xd segments, key: <id_xd>; value: dict() with key "id_tmc"

# with open("../data/tmc_to_xd.pkl", "wb") as f1,  open("../data/xd_to_tmc.pkl", "wb") as f2:
#     pickle.dump(tmc_to_xd, f1)
#     pickle.dump(xd_to_tmc, f2)

In [7]:
with open("../data/tmc_to_xd.pkl", "rb") as f1,  open("../data/xd_to_tmc.pkl", "rb") as f2:
    tmc_to_xd = pickle.load(f1)
    xd_to_tmc = pickle.load(f2)

### 1.2 Get Downstream Segments within 5 Miles (used for incident feature processing)

In [64]:
# stored as dict() objects
# key: <id_tmc> or <id_xd>
# value: list of downstream segments
with open("../data/next_tmc_5_miles.pkl", "rb") as f1, open("../data/next_xd_5_miles.pkl", "rb") as f2:
    next_tmc = pickle.load(f1)  # 315 tmc segments in Cranberry, covering all new_in_tmc
    next_xd = pickle.load(f2)  # 117679 xd segments in PA, covering all new_in_xd

### 1.3 Select Input & Output Segments

In [10]:
new_in_seg = {}
new_in_seg["tti"] = {}
new_in_seg["tti"]["tmc"] = set(tmc[tmc["tmc"].isin(old_in_seg["tti"]["tmc"])]["tmc"]).intersection(tmc_xd_join.id_tmc) # 146, including all new_out_tmc
new_in_seg["tti"]["xd"] = set(xd[xd.id_xd_str.isin(old_in_seg["tti"]["xd"])]["id_xd_str"]).intersection(tmc_xd_join.id_xd_int.apply(str)) # 8, including all new_out_xd
new_in_seg["inc"] = {}
new_in_seg["inc"]["tmc"] = set(tmc[tmc["tmc"].isin(old_in_seg["inc"]["tmc"])]["tmc"]).intersection(tmc_xd_join.id_tmc) # 146, including all new_out_tmc
new_in_seg["inc"]["xd"] = set(xd[xd.id_xd_str.isin(old_in_seg["inc"]["xd"])]["id_xd_str"]).intersection(tmc_xd_join.id_xd_int.apply(str)) # 8, including all new_out_xd
new_in_seg["sd"] = {}
new_in_seg["sd"]["tmc"] = set(tmc[tmc["tmc"].isin(old_in_seg["sd"])]["tmc"]).intersection(tmc_xd_join.id_tmc) # 142, including all new_out_tmc
new_in_seg["sd"]["xd"] = set(xd[xd.id_xd_str.isin(old_in_seg["sd"])]["id_xd_str"]).intersection(tmc_xd_join.id_xd_int.apply(str)) # 8, including all new_out_xd

In [11]:
new_in_tmc = set().union(new_in_seg["tti"]["tmc"], new_in_seg["sd"]["tmc"], new_in_seg["inc"]["tmc"])  # 146
new_in_xd = set().union(new_in_seg["tti"]["xd"], new_in_seg["sd"]["xd"], new_in_seg["inc"]["xd"])  # 8
new_in_xd_int = set([int(i) for i in list(new_in_xd)])
new_in = new_in_tmc.union(new_in_xd)  # 154 targeted TMC & XD segments for input

In [12]:
new_in_all_in_xd_int = set(new_in_xd_int) # 260 xd id in int
for id_tmc in list(new_in_tmc):
    new_in_all_in_xd_int = new_in_all_in_xd_int.union(set(tmc_to_xd[id_tmc]["id_xd_int"]))

new_in_all_in_xd_str = set([str(i) for i in list(new_in_all_in_xd_int)])

In [13]:
new_in_all_in_tmc = new_in_tmc  # 146 tmc segments
for id_xd in list(new_in_xd):
    id_xd = int(id_xd)
    new_in_all_in_tmc = new_in_all_in_tmc.union(set(xd_to_tmc[id_xd]["id_tmc"]))

In [15]:
new_all_in = new_in_all_in_tmc.union(new_in_all_in_xd_str)  # 146 TMC & 260 XD segments for input

In [16]:
new_out_tmc = set(tmc[tmc.tmc.isin(old_out)]["tmc"]).intersection(tmc_xd_join.id_tmc)  # 67 preliminarily selected TMC segments for output
new_out_xd = set(xd[xd.id_xd_str.isin(old_out)]["id_xd_str"]).intersection(tmc_xd_join.id_xd_int.apply(str))  # 8 preliminarily selected XD segments for output
new_out_xd_int = set([int(i) for i in list(new_out_xd)])

In [17]:
new_out_all_in_xd_int = set(new_out_xd_int) # 136 xd segments in int
for id_tmc in list(new_out_tmc):
    new_out_all_in_xd_int = new_out_all_in_xd_int.union(set(tmc_to_xd[id_tmc]["id_xd_int"]))

new_out_all_in_xd_str = set([str(i) for i in list(new_out_all_in_xd_int)])

In [18]:
new_out_all_in_tmc = new_out_tmc  # 67 tmc segments
for id_xd in list(new_out_xd):
    id_xd = int(id_xd)
    new_out_all_in_tmc = new_out_all_in_tmc.union(set(xd_to_tmc[id_xd]["id_tmc"]))

In [19]:
new_all_out = new_out_all_in_tmc.union(new_out_all_in_xd_str)

## Part 2. Generate New Input & Output Data

In [20]:
# Dates and Timewindows that we are interested in 
start_date = dt(2019, 2, 10)
end_date = dt(2019, 7, 23)
start_min = 330 # 05:30:00 
end_min = 1260 # 21:00:00
busi_date = pd.bdate_range(start=start_date, end=end_date).date  # 117, note that the end_date here is inclusive 
all_date = pd.date_range(start=start_date, end=end_date).date # 164 dates, note that the end_date here is inclusive 
busi_idx = [i for i, b in enumerate(all_date) if b in busi_date]

### 2.1 Load Data

#### 2.1.1 Load Old X and Old Y

In [21]:
'''
Dates of old X & old Y: 2019.2.10 ~ 2019.7.23 (164 days, including all holidays & weekends)
Time Slots: 
    - For each day, there are 180 targeted time slots from 06:00:00 to 20:55:00
    - For each targeted time slot t (t in 06:00:00 ~ 20:55:00), 
        - old_Y contains padding of 7 slots (t-6, t-5, t-4, t-3, t-2, t-1, t)
        - old_X contains padding of 7 slots as input (t-12, t-11, t-10, t-9, t-8, t-7, t-6)
    
    In new_X and new Y, to allow for more flexibility of hyperparameters and reduce the file size, there won't be padding
    For example, for targeted time slot 06:00:00
        - old_Y has 05:30:00, 05:35:00, 05:40:00, 05:45:00, 05:50:00, 05:55:00, 06:00:00
        - old_X has 05:00:00, 05:05:00, 05:10:00, 05:15:00, 05:20:00, 05:25:00, 05:30:00
        - new_Y has 06:00:00
        - new_X has 05:30:00
'''
old_X = np.load("../data/X.npy")  # (29520, 7, 1008)
old_Y = np.load("../data/Y.npy")  # (29520, 7, 162)
old_col = list(np.load("../data/col_names.npy", allow_pickle=True))  # 1008 (369 tti, 315 inc, 303 sd, 21 weather & time)

#### 2.1.2 Load TMC Data

In [24]:
'''
Columns:
    'tmc_code', 'measurement_tstamp', 'speed', 'average_speed',
    'reference_speed', 'travel_time_minutes', 'data_density'
'''
tmc_truck = pd.read_csv("../data/Carnberry_NPMRDS_5min/manually_select_cranberry_2019_dont_average/manually_select_cranberry_2019_dont_average.csv") # 9136192, 7
tmc_pv = pd.read_csv("../data/Carnberry_NPMRDS_5min/manually_select_cranberry_2019_dont_average_3/manually_select_cranberry_2019_dont_average.csv")  # 21385940, 7
tmc_all = pd.read_csv("../data/Carnberry_NPMRDS_5min/manually_select_cranberry_2019_dont_average_2/manually_select_cranberry_2019_dont_average.csv") # 24388983,7

#### 2.1.3 Load XD Data

run the next block ONLY IF `../data/downsampled_xd_data.csv` is not in place

In [26]:
'''
Header:
    'xd_id', 'measurement_tstamp', 'speed', 'average_speed', 'reference_speed', 'travel_time_minutes', 'confidence_score', 'cvalue'
'''
# the original csv file stores XD speed data in 1-min slots from 2018.11.1 to 2019.7.27, which is too large
# therefore, we have to read and split csv into 61 dataframe chunks and apply operation individually 
chunksize = 10 ** 7
xd_file = "../data/Cranberry_ritis_1min_class123/manually_select_cranberry_class123_20181101_20190727_dont_average/manually_select_cranberry_class123_20181101_20190727_dont_average.csv"
chunklist = []
with pd.read_csv(xd_file, chunksize=chunksize) as reader:
    for chunk in tqdm(reader):
        chunk.measurement_tstamp = pd.to_datetime(chunk.measurement_tstamp)

        # filter dataframe by selecting rows based on xd_id and timestamp of our interest
        # here we select chunk from 05:30:00 to 20:59:00 to accomodate time range for both input feature (05:30:00 - 20:25:00, 5-min frequency) and output ground truth (06:00:00 - 20:59:00, 1-min frequency)
        chunk = chunk[
                (chunk.xd_id.isin(new_in_all_in_xd_int)) &
                (chunk.measurement_tstamp.dt.date.isin(busi_date)) & 
                (chunk.measurement_tstamp.dt.hour*60 + chunk.measurement_tstamp.dt.minute >= 330) & 
                (chunk.measurement_tstamp.dt.hour*60 + chunk.measurement_tstamp.dt.minute < 1260) 
                ]
        chunklist.append(chunk)

# concat dataframe chunks and merge into one final dataframe 
downsampled_xd = pd.concat(chunklist) 
downsampled_xd = downsampled_xd.reset_index(drop=True)  # reset index

# save downsampled xd data
downsampled_xd.to_csv("../data/downsampled_xd_data.csv", index=False)

62it [06:02,  5.84s/it]


In [27]:
downsampled_xd = pd.read_csv("../data/downsampled_xd_data.csv")  # (260 xd * 109740 time slots, 8)
downsampled_xd.measurement_tstamp = pd.to_datetime(downsampled_xd.measurement_tstamp)

### 2.2 Generate New Input and Output

#### 2.2.1 Prepare Density Feature for Input

2.1.1.1 TMC Density

In [28]:
density_tmc = tmc_all.loc[:, ["tmc_code", "measurement_tstamp", "data_density"]]
density_tmc = density_tmc.pivot(index = "measurement_tstamp", columns = "tmc_code", values = "data_density")

# select 143 tmc segments based on new input segments in terms of TMC, which also includes all new_out_tmc
density_tmc = density_tmc.loc[:, [c for c in density_tmc.columns if c in new_in_all_in_tmc]] 

# fill NAN with "A", which denotes "Fewer than five values"
density_tmc = density_tmc.fillna("A")

# convert index to datetime object, and select 21060 rows of interest
density_tmc.index = pd.to_datetime(density_tmc.index)
density_tmc = density_tmc[pd.Index(density_tmc.index.date).isin(busi_date)]
density_tmc = density_tmc[(density_tmc.index.hour * 60 + density_tmc.index.minute >= 330 ) & (density_tmc.index.hour * 60 + density_tmc.index.minute <= 1225)]  # (117 * 180, 143)

# ordinal embedding 
density_tmc = density_tmc.replace(["A", "B", "C"], [1/6, 3/6, 5/6]) # (21060, 143)

2.1.1.2 XD Density

In [29]:
density_xd = downsampled_xd.pivot(index = "measurement_tstamp", columns = "xd_id", values = "confidence_score")  # (117 * 930, 260)

# select 260 xd segments based on new input segments in terms of XD, which also includes all new_out_xd
density_xd = density_xd.loc[:, [c for c in density_xd.columns if c in new_in_all_in_xd_int]] 

# select date and time range
density_xd = density_xd[pd.Index(density_xd.index.date).isin(busi_date)]
density_xd = density_xd[(density_xd.index.hour*60 + density_xd.index.minute >= 330) & 
                (density_xd.index.hour*60 + density_xd.index.minute < 1230) 
                ].reset_index() 

# fill null
density_xd = density_xd.fillna(10.)

# ordinal embedding
density_xd = density_xd.replace([10., 20., 30.], [1/6, 3/6, 5/6])

# aggregation
density_xd = density_xd.groupby(density_xd.index // 5).mean()  # (21060, 260)

#### 2.2.2 Prepare Speed Data for Input and Output

2.2.2.1 TMC Speed (All, Truck, Personal Vehicle)

Note - Here I will first extract speed data from 05:30:00~20:55:00, and then extract input feature (05:30:00~20:25:00) and output ground truth (06:00:00~20:55:00)

In [30]:
# speed of all vehicles
all_spd = tmc_all.loc[:, ["tmc_code", "measurement_tstamp", "speed"]]
all_spd = all_spd.pivot(index = "measurement_tstamp", columns = "tmc_code", values = "speed")

# select 143 tmc segments based on all tmc input segments
all_spd = all_spd.loc[:, [c for c in all_spd.columns if c in new_in_all_in_tmc]]  

# convert index to datetime object, and select 21762 rows of interest
all_spd.index = pd.to_datetime(all_spd.index)
all_spd = all_spd[pd.Index(all_spd.index.date).isin(busi_date)]
all_spd = all_spd[(all_spd.index.hour * 60 + all_spd.index.minute >= 330 ) & (all_spd.index.hour * 60 + all_spd.index.minute <= 1255)] # (117 * 186, 143) covers 05:30:00~20:55:00, 737664 NaN

In [31]:
truck = tmc_truck.loc[:, ["tmc_code", "measurement_tstamp", "speed"]]
truck = truck.pivot(index = "measurement_tstamp", columns = "tmc_code", values = "speed")

# select 143 tmc segments based on old tmc input segments
truck = truck.loc[:, [c for c in truck.columns if c in new_in_all_in_tmc]]  

# convert index to datetime object, and select 21762 rows of interest
truck.index = pd.to_datetime(truck.index)
truck = truck.resample("5 min").asfreq()  # upsampling with 5-min frequency (otherwise, there will be 41 time slots missing)
truck = truck[pd.Index(truck.index.date).isin(busi_date)]
truck = truck[(truck.index.hour * 60 + truck.index.minute >= 330 ) & (truck.index.hour * 60 + truck.index.minute <= 1255)] # (117 * 186, 143), 1827359 NaN

In [32]:
pv = tmc_pv.loc[:, ["tmc_code", "measurement_tstamp", "speed"]]
pv = pv.pivot(index = "measurement_tstamp", columns = "tmc_code", values = "speed")

# select 143 tmc segments based on old tmc input segments
pv = pv.loc[:, [c for c in pv.columns if c in new_in_all_in_tmc]] 

# convert index to datetime object, and select 21762 rows of interest
pv.index = pd.to_datetime(pv.index)
pv = pv[pd.Index(pv.index.date).isin(busi_date)]
pv = pv[(pv.index.hour * 60 + pv.index.minute >= 330 ) & (pv.index.hour * 60 + pv.index.minute <= 1255)]  # (117 * 186, 143), 895710 NaN

In [34]:
# fillna with latest observations
tmc_all = all_spd.fillna(method='ffill')  # 406 NaN
tmc_truck = truck.fillna(method='ffill')  # 15350 NaN
tmc_pv = pv.fillna(method='ffill')  # 479 NaN

tmc_all = tmc_all.fillna(method='bfill')  # 0 NaN
tmc_truck = tmc_truck.fillna(method='bfill')  # 0 NaN
tmc_pv = tmc_pv.fillna(method='bfill')  # 0 NaN

In [35]:
# INPUT

tmc_all_in = tmc_all[(tmc_all.index.hour * 60 + tmc_all.index.minute >= 330 ) & (tmc_all.index.hour * 60 + tmc_all.index.minute <= 1225)]  #(21060, 143)
tmc_truck_in = tmc_truck[(tmc_truck.index.hour * 60 + tmc_truck.index.minute >= 330 ) & (tmc_truck.index.hour * 60 + tmc_truck.index.minute <= 1225)]  #(21060, 143)
tmc_pv_in = tmc_pv[(tmc_pv.index.hour * 60 + tmc_pv.index.minute >= 330 ) & (tmc_pv.index.hour * 60 + tmc_pv.index.minute <= 1225)]  #(21060, 143)

In [36]:
# OUTPUT

tmc_all_out = tmc_all[(tmc_all.index.hour * 60 + tmc_all.index.minute >= 360 ) & (tmc_all.index.hour * 60 + tmc_all.index.minute <= 1255)]  #(21060, 233)
tmc_truck_out = tmc_truck[(tmc_truck.index.hour * 60 + tmc_truck.index.minute >= 360 ) & (tmc_truck.index.hour * 60 + tmc_truck.index.minute <= 1255)]  #(21060, 233)
tmc_pv_out = tmc_pv[(tmc_pv.index.hour * 60 + tmc_pv.index.minute >= 360 ) & (tmc_pv.index.hour * 60 + tmc_pv.index.minute <= 1255)]  #(21060, 233)

tmc_all_out = tmc_all_out[new_out_tmc]  # (21060, 67)
tmc_truck_out = tmc_truck_out[new_out_tmc]  # (21060, 67)
tmc_pv_out = tmc_pv_out[new_out_tmc]  # (21060, 67)

/var/folders/y2/1kk0np3x01qchz21plxx_m0w0000gn/T/ipykernel_10958/821678476.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  tmc_all_out = tmc_all_out[new_out_tmc]  # (21060, 67)
/var/folders/y2/1kk0np3x01qchz21plxx_m0w0000gn/T/ipykernel_10958/821678476.py:8: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  tmc_truck_out = tmc_truck_out[new_out_tmc]  # (21060, 67)
/var/folders/y2/1kk0np3x01qchz21plxx_m0w0000gn/T/ipykernel_10958/821678476.py:9: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  tmc_pv_out = tmc_pv_out[new_out_tmc]  # (21060, 67)


2.2.2.2 XD Speed Feature

In [37]:
# INPUT

xd_spd_in = downsampled_xd.pivot(index = "measurement_tstamp", columns = "xd_id", values = "speed")  # (117 * 930, 260), 1-min frequency, 621745 NaN

# forward filling
xd_spd_in = xd_spd_in.fillna(method="ffill") # no NaN

# select the appropriate time range for output and input 
xd_spd_in = xd_spd_in[pd.Index(xd_spd_in.index.date).isin(busi_date)]
xd_spd_in = xd_spd_in[(xd_spd_in.index.hour*60 + xd_spd_in.index.minute >= 330) & 
                (xd_spd_in.index.hour*60 + xd_spd_in.index.minute < 1230)].reset_index()   # for input, select 05:30:00 ~ 20:25:00
xd_spd_in =  xd_spd_in.groupby(xd_spd_in.index // 5).mean()  # 5-min frequency, (21060, 260), no NAN => used as XD speed input feature

In [38]:
# OUTPUT

xd_spd_out = downsampled_xd.pivot(index = "measurement_tstamp", columns = "xd_id", values = "speed")  # (109740, 260), 1-min frequency
xd_spd_out = xd_spd_out.interpolate(method="linear") # no NAN

# select the appropriate time range for output and input 
xd_spd_out = xd_spd_out[pd.Index(xd_spd_out.index.date).isin(busi_date)]
xd_spd_out = xd_spd_out[(xd_spd_out.index.hour*60 + xd_spd_out.index.minute >= 360) & 
                (xd_spd_out.index.hour*60 + xd_spd_out.index.minute < 1260) 
                ].reset_index()   # for output, select 06:00:00 ~ 20:59:00 => 1-min frequency, (147600, 69), no NAN => used as XD speed output ground truth

# rearrange column order, and add the missing column to make 70 columns in total
xd_spd_out = xd_spd_out[new_out_xd_int]  # (106200, 8), same column order as new_out_all_in_xd_int
xd_spd_out =  xd_spd_out.groupby(xd_spd_out.index // 5).mean() # 5-min frequency, (21060, 8), no NAN => used as XD speed output ground truth

/var/folders/y2/1kk0np3x01qchz21plxx_m0w0000gn/T/ipykernel_10958/4218426228.py:13: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  xd_spd_out = xd_spd_out[new_out_xd_int]  # (106200, 8), same column order as new_out_all_in_xd_int


#### 2.2.3 Prepare Incident Data for Input and Output

2.2.3.1 Incident Input Feature

In [39]:
# Step 1. Load Old X

# get indices of old_col that will remain as new columns
col_idx = []  # will store the indices of 479 old columns that will become new_X (146+8 tti, 150 sd, 154 inc, 21 weather & time)
for i in range(987):
    c = old_col[i]
    if "sd" in c:
        c = c[3:]
    if "inc" in c:
        c = c[4:]
    
    if c in new_all_in:
        col_idx.append(i)
col_idx += list(range(987, 1008))

In [40]:
# new_X without additional features
# tti_tmc will be column 0:146
# tti_xd will be column 146:154
# inc_tmc will be column 304:450
# inc_xd will be column 450:458
# weather: 458:479
new_X = old_X[:, -1, col_idx].reshape(164, 180, -1) # (164, 180, 479) 164 days (busi + non-busi) * 180 daily time slots (05:30:00 ~ 20:25:00)

# select business dates only
new_X = new_X[busi_idx, :, :].reshape(-1, 479)  # (117*180, 479)

In [41]:
# Step 2. Mark Segments with Abnormal Speed as in Incident

# check abnormal speed
in_abnormal_level = 0.25
tmc_all_in_abnormal = (tmc_all_in < tmc_all_in.quantile(q=in_abnormal_level, axis=0)).reset_index()  # (21060, 144)
xd_spd_in_abnormal = xd_spd_in < xd_spd_in.quantile(q=in_abnormal_level, axis=0) # (21060, 260)

In [42]:
inc_in_id_tmc = [i for i in old_in_seg["inc"]["tmc"] if i in new_all_in] # 146
inc_in_id_xd_str = [i for i in old_in_seg["inc"]["xd"] if i in new_all_in]  # 8, in str

In [43]:
xd_spd_in_abnormal

xd_id,134149761,134357035,134452998,135120710,388033516,388051958,389154131,389249646,389250331,389254810,...,1310596992,1310597003,1310597255,1310597301,1310597378,1310597390,1310597509,1310604066,1310609751,1310614041
0,False,True,True,False,True,False,True,False,True,False,...,False,True,False,True,True,True,False,False,True,False
1,False,True,True,True,True,False,True,False,False,False,...,False,True,False,True,False,True,False,False,True,True
2,False,True,True,False,True,False,True,False,False,True,...,False,True,False,True,False,False,False,True,True,True
3,False,False,True,False,True,False,True,False,False,True,...,True,True,False,True,True,False,False,False,True,False
4,False,False,False,False,False,False,True,False,True,False,...,False,True,False,True,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21055,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
21056,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,False
21057,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,False
21058,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True


In [44]:
tmc_all_in_abnormal

tmc_code,measurement_tstamp,104+04440,104+04441,104+04442,104+04443,104+04444,104+04445,104+04528,104+04530,104+04531,...,104P04734,104P04740,104P04742,104P04782,104P04784,104P06223,104P06230,104P06676,104P11466,104P13684
0,2019-02-11 05:30:00,True,True,True,True,True,False,True,True,True,...,False,True,False,False,False,False,False,False,False,True
1,2019-02-11 05:35:00,True,True,True,True,True,False,True,True,False,...,False,True,False,False,False,False,False,False,True,True
2,2019-02-11 05:40:00,False,False,False,False,True,True,True,True,False,...,False,True,False,False,False,False,False,False,True,True
3,2019-02-11 05:45:00,False,False,False,False,False,False,True,True,True,...,False,True,False,True,True,False,False,False,False,True
4,2019-02-11 05:50:00,False,False,True,True,False,False,True,True,True,...,False,True,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21055,2019-07-23 20:05:00,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
21056,2019-07-23 20:10:00,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
21057,2019-07-23 20:15:00,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
21058,2019-07-23 20:20:00,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [45]:
for id in tqdm(tmc_all_in_abnormal.columns):
    for t in tmc_all_in_abnormal[id][tmc_all_in_abnormal[id] == True].index:
        idx = inc_in_id_tmc.index(id) + 304
        new_X[t, idx] = max(new_X[t, idx], 1)

100%|██████████| 144/144 [00:00<00:00, 174.85it/s]


In [46]:
for i, id in tqdm(enumerate(inc_in_id_xd_str)):
    for t in xd_spd_in_abnormal[int(id)][xd_spd_in_abnormal[int(id)] == True].index:
        idx = i + 450
        new_X[t, idx] = max(new_X[t, idx], 1)

8it [00:00, 395.16it/s]


In [47]:
# Ordinal Embedding for Inc Features
new_X[:,304:458][new_X[:,304:458] == 0.0] = 1/6
new_X[:,304:458][new_X[:,304:458] == 1.0] = 3/6
new_X[:,304:458][new_X[:,304:458] == 2.0] = 5/6

2.2.3.2 Incident Output Ground Truth

In [49]:
# Step 1. Load Incident Data

out_inc_label = pd.read_csv("../data/incident_labels.csv")  # 5-min frequency
out_inc_label = out_inc_label.set_index("measurement_tstamp")
out_inc_label.index = pd.to_datetime(out_inc_label.index)

In [50]:
inc_out_id_tmc = [i for i in out_inc_label.columns if i in new_out_tmc]  # 67
inc_out_id_xd_str = [i for i in out_inc_label.columns if i in new_out_xd]  # 8

In [51]:
out_inc_label

,104+04438,104+04439,104+04440,104+04441,104+04442,104+04443,104+04444,104+04445,104+04528,104+04529,...,1310574747,1310574829,1310583799,1310593380,1310597255,1310597366,1310597390,1310597496,1310597520,389134964
measurement_tstamp,,,,,,,,,,,,,,,,,,,,,
2019-02-10 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 00:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 00:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07-23 23:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-23 23:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-23 23:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
# select rows based on timestamps of our interest (2019.2.10 ~ 2019.7.23, 06:00:00~20:55:00 in 5-min frequency everyday)
out_inc_5_min = out_inc_label[
                (pd.Index(out_inc_label.index.date).isin(busi_date)) &
                (out_inc_label.index.hour*60 + out_inc_label.index.minute >= 360) & 
                (out_inc_label.index.hour*60 + out_inc_label.index.minute < 1260) 
                ]

# select columns of new out segments 
out_inc_5_min = out_inc_5_min.loc[:, inc_out_id_tmc+inc_out_id_xd_str].reset_index(drop=True)  # (21060, 75), 20034 events

# rename columns to represent all segments in TMC IDs, so that we can add out_inc_5_min and speed_outliers_5_min
# out_inc_5_min.columns = speed_outliers_5_min.columns 

In [53]:
# Step 2. Mark Segments with Abnormal Speed as in Incident
# check abnormal speed
out_abnormal_level = 0.25
tmc_all_out_abnormal = (tmc_all_out < tmc_all_out.quantile(q=out_abnormal_level, axis=0)).reset_index()  # (21060, 144)
xd_spd_out_abnormal = xd_spd_out < xd_spd_out.quantile(q=out_abnormal_level, axis=0) # (21060, 260)

In [54]:
for id in tqdm(out_inc_5_min.columns):
    if id in inc_out_id_tmc:
        for t in tmc_all_out_abnormal[id][tmc_all_out_abnormal[id] == True].index:
            out_inc_5_min.loc[t, id] = max(out_inc_5_min.loc[t, id], 1)
    else:
        for t in xd_spd_out_abnormal[int(id)][xd_spd_out_abnormal[int(id)] == True].index:
            out_inc_5_min.loc[t, id] = max(out_inc_5_min.loc[t, id], 1)

100%|██████████| 75/75 [00:10<00:00,  6.92it/s]


In [57]:
# Step 3. Mark Downstream Segments of Segments with Abnormal Speed Segements as in Incident

In [71]:
for id in tqdm(out_inc_5_min.columns):
    if id in inc_out_id_tmc:
        for time in tmc_all_out_abnormal[id][tmc_all_out_abnormal[id] == True].index:
            date = time // 180
            slot = time % 180
            downstream_tmc = next_tmc[id]
            for d in downstream_tmc:
                if d not in inc_out_id_tmc:
                    continue
                for t in range(time, date*180 + min(180, slot+8)):
                    out_inc_5_min.loc[t, d] = max(out_inc_5_min.loc[t, d], 1)
    else:
        for time in xd_spd_out_abnormal[int(id)][xd_spd_out_abnormal[int(id)] == True].index:
            date = time // 180
            slot = time % 180
            # downstream_xd = next_xd[str(id)]
            downstream_xd = next_xd[int(id)]
            for d in downstream_xd:
                d = str(d)
                if d not in inc_out_id_xd_str:
                    continue
                for t in range(time, date*180 + min(180, slot+8)):
                    out_inc_5_min.loc[t, d] = max(out_inc_5_min.loc[t, d], 1)

100%|██████████| 75/75 [01:06<00:00,  1.13it/s]


In [72]:
sum(out_inc_5_min[out_inc_5_min>0].sum())  # 941026 events

941026.0

#### 2.2.4 Merge New Features into New_X

In [73]:
# (21060, 1571) 
# second dimension: 
#       403 density (143 tmc + 260 xd)
#       689 raw speed (260 xd spd, 143 tmc spd all, 143 tmc spd truck, 143 tmc spd pv) 
#       154 tti
#       150 sd
#       154 inc
#       21 weather & time
final_X = np.concatenate((density_tmc.to_numpy(), density_xd.to_numpy(), xd_spd_in.to_numpy(), tmc_all_in.to_numpy(), tmc_truck_in.to_numpy(), tmc_pv_in.to_numpy(), new_X), axis=1) 

In [74]:
# Scaling Normalization (min-max normalization)
scaler = MinMaxScaler()
final_X = scaler.fit_transform(final_X)

In [75]:
np.save("../data/new_X_v2.npy", final_X)

#### 2.2.5 Merge Ground Truth Data into New_Y

In [77]:
# step 1. Merge TMC Speed & XD Speed
new_Y_spd = np.concatenate((tmc_all_out, xd_spd_out), axis=1)  # (21060, 75)

# step 2. Stack Speed Data with Incident Status Data
new_Y = np.stack((new_Y_spd, out_inc_5_min.to_numpy().astype("float64")), axis=-1)  # (21060, 75, 2)

In [78]:
np.save("../data/new_Y_v2.npy", new_Y)

## Part 3 Waze DataFrame

Generate Waze report dataframe for segments selected for output. The dataframe will be used to measure timeliness of incident status prediction by our 2-stage model.

In [55]:
'''
size (47520, 315)
47520: 165 days (2019.2.10 ~ 2019.7.24) * 24 hrs * 12 slots/hr
'''
waze_df = pd.read_pickle("../data/waze_df.pickle")

In [56]:
waze_out = waze_df[new_out_tmc.union(new_out_xd)]
waze_out_5_min = waze_out[
            (pd.Index(waze_out.index.date).isin(busi_date)) & 
            (waze_out.index.hour*60 + waze_out.index.minute >= 360) & 
            (waze_out.index.hour*60 + waze_out.index.minute < 1260)
            ]  # (21060, 75), with same segment order as new_Y_tmc and new_Y_xd

/var/folders/y2/1kk0np3x01qchz21plxx_m0w0000gn/T/ipykernel_6888/2191235339.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  waze_out = waze_df[new_out_tmc.union(new_out_xd)]


In [57]:
waze_out_5_min

,104+04736,104N04441,104P06676,104-04442,104P04441,104-04742,1310597255,104P04444,104+06681,1310330692,...,1310427817,104-04739,104+06678,104-04542,104+06680,104-06680,1310457035,104N04442,104-11466,104-04444
2019-02-11 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-11 06:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-11 06:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-11 06:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-11 06:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07-23 20:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-23 20:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-23 20:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-23 20:50:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
np.save("../data/waze_out_5_min_v2.npy", waze_out_5_min.to_numpy())